In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.transform
import skimage.filters
import skimage.feature
import skimage.restoration
from skimage.transform import resize
from skimage.filters import sobel
from skimage.feature import canny
from skimage.restoration import denoise_bilateral, denoise_tv_chambolle
from skimage.io import imread, imshow
from sklearn.cluster import KMeans
from scipy import misc
from skimage.color import rgb2gray
from skimage.io import imread, imshow
import shutil
import glob

import boto3
import sys
sys.path.insert(0,'/home/newton/images')
import os
import pickle
import dill

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
# tf.enable_eager_execution()

import tensorflow_hub as hub
# import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
import tensorflow.keras.experimental
import keras.applications.mobilenet
from keras.preprocessing import image
import file_image_processor

Using TensorFlow backend.


In [2]:
base_dir= '/home/newton/images/facehair_train'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
batch_size = 100
IMG_W = 224
IMG_H = 224

In [3]:
image_gen_train = ImageDataGenerator(rescale=1/255)
train_data_gen = image_gen_train.flow_from_directory(
                                                batch_size=batch_size, 
                                                directory=train_dir, 
                                                shuffle=True, 
                                                target_size=(IMG_W,IMG_H),
                                                class_mode='sparse'
                                                )

Found 6400 images belonging to 2 classes.


In [4]:
image_gen_val = ImageDataGenerator(rescale=1/255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size, 
                                                 directory=val_dir, 
                                                 target_size=(IMG_W, IMG_H),
                                                 class_mode='sparse')

Found 1600 images belonging to 2 classes.


In [5]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"

feature_extractor = hub.KerasLayer(URL)

feature_extractor.trainable = False

m3_facehair = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(5, activation='softmax')
])

In [6]:
m3_facehair.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
epochs = 15

history = m3_facehair.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen
)

Epoch 1/15
64/64 [==============================] - 222s 3s/step - loss: 0.5165 - acc: 0.7456 - val_loss: 0.5538 - val_acc: 0.7181
Epoch 2/15
64/64 [==============================] - 221s 3s/step - loss: 0.5076 - acc: 0.7522 - val_loss: 0.5506 - val_acc: 0.7131
Epoch 3/15
64/64 [==============================] - 220s 3s/step - loss: 0.4993 - acc: 0.7545 - val_loss: 0.5482 - val_acc: 0.7244
Epoch 4/15
64/64 [==============================] - 221s 3s/step - loss: 0.4939 - acc: 0.7619 - val_loss: 0.5568 - val_acc: 0.7156
Epoch 5/15
64/64 [==============================] - 220s 3s/step - loss: 0.4880 - acc: 0.7636 - val_loss: 0.5533 - val_acc: 0.7250
Epoch 6/15
64/64 [==============================] - 220s 3s/step - loss: 0.4812 - acc: 0.7713 - val_loss: 0.5487 - val_acc: 0.7188
Epoch 7/15
64/64 [==============================] - 220s 3s/step - loss: 0.4794 - acc: 0.7731 - val_loss: 0.5487 - val_acc: 0.7237
Epoch 8/15
64/64 [==============================] - 219s 3s/step - loss: 0.4822 - a

In [9]:
m3_facehair.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  2257984   
_________________________________________________________________
dense (Dense)                multiple                  6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [10]:
m3_facehair.save('./models/facehair_model_1.h5')

In [2]:
new_model = tf.keras.models.load_model('./models/facehair_model_1.h5',custom_objects={'KerasLayer':hub.KerasLayer})

In [3]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  2257984   
_________________________________________________________________
dense (Dense)                multiple                  6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
# prediction code below

In [8]:
im = prepare_image(os.path.join('/home/newton/images/gender_images/train/female','000001.jpg'))

In [9]:
result = new_model.predict(im)

In [ ]:
#  the results seem to be that the first number is glasses, the second is no glasses.
#  The other three are irrelevant.
#  the image used here has glasses.

In [10]:
print (result)

[[6.8240178e-01 3.1759244e-01 3.4340760e-06 1.8126588e-06 5.8129763e-07]]
